In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi
from ashn import *
from ashn.utils import canonical_decompose
import cirq
from pytket.circuit import Op, OpType
from scipy import linalg
from rich.console import Console

console = Console()

I = np.identity(2).astype(complex)
X = Op.create(OpType.X).get_unitary()
Y = Op.create(OpType.Y).get_unitary()
Z = Op.create(OpType.Z).get_unitary()
XX = np.kron(X, X)
YY = np.kron(Y, Y)
ZZ = np.kron(Z, Z)

In [15]:
iswap_family_coords = [np.array((x, x, 0)) for x in [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]]
iswap_family = [Op.create(OpType.TK2, coord) for coord in iswap_family_coords]

In [16]:
iswap_family[3]

TK2(0.25, 0.25, 0)

In [17]:
g = 6.25 * 2 * pi  # e.g., assume the unit is MHz
a, b, c = g/2, g/2, 0

In [18]:
# iSWAP gate
print(iswap_family[3])
coord = iswap_family_coords[3]
tau, Omega1, Omega2, delta = solve_ashn(*(iswap_family_coords[3] * pi / 2), a, b, c)
print('tau: {:.4f}, Omega1: {:.4f}, Omega2: {:.4f}, delta: {:.4f}'.format(tau, Omega1, Omega2, delta))

TK2(0.25, 0.25, 0)
>>> Using AshN-ND <<<
sinc_S1tau: 0.9744953584044326, sinc_S2tau: 0.9744953584044326
tau: 0.0200, Omega1: 0.0000, Omega2: 0.0000, delta: 0.0000


In [19]:
def obtain_single_qubit_corrections(coord, coeffs, tau, Omega1, Omega2, delta):
    """Verify AshN solution"""
    a1_, a2_ = I, I
    b1_, b2_ = I, I
    a, b, c = coeffs
    x, y, z = coord
    O1 = I
    O2 = I
    H1_, H2_ = np.zeros([2, 2]), np.zeros([2, 2])
    H = np.kron(H1_, I) + np.kron(I, H2_) + np.kron(O1, O2) @ (a * XX + b * YY + c * ZZ) @ np.kron(O1, O2).conj().T
    H1__ = (Omega1 + Omega2) * X + delta * Z
    H2__ = (Omega1 - Omega2) * X + delta * Z
    H1, H2 = O1 @ H1__ @ O1.conj().T - H1_, O2 @ H2__ @ O2.conj().T - H2_
    (a1__, a2__), (b1__, b2__), coord_ = canonical_decompose(
        linalg.expm(-1j * tau * (a * XX + b * YY + c * ZZ + np.kron(H1__, I) + np.kron(I, H2__))))
    a1, a2 = a1_ @ a1__.conj().T @ O1.conj().T, a2_ @ a2__.conj().T @ O2.conj().T
    b1, b2 = O1 @ b1__.conj().T @ b1_, O2 @ b2__.conj().T @ b2_

    return (a1, a2), (b1, b2)

    print('Original Weyl coord: {}'.format(coord))
    print('Solved Weyl coord: {}'.format(coord_))
    print('Correct?:', np.allclose(coord_, coord))

    v = np.kron(a1_, a2_) @ linalg.expm(-1j * (x * XX + y * YY + z * ZZ)) @ np.kron(b1_, b2_)
    u = np.kron(a1, a2) @ linalg.expm(-1j * tau * (H + np.kron(H1, I) + np.kron(I, H2))) @ np.kron(b1, b2)

    cirq.testing.assert_allclose_up_to_global_phase(u, v, atol=1e-6)


In [20]:
single_qubit_corrections = {
    'after_1': [],
    'after_2': [],
    'before_1': [],
    'before_2': [],
}
for coord in iswap_family_coords:
    tau, Omega1, Omega2, delta = solve_ashn(*(coord * pi / 2), a, b, c)
    print('tau: {:.4f}, Omega1: {:.4f}, Omega2: {:.4f}, delta: {:.4f}'.format(tau, Omega1, Omega2, delta))
    (a1, a2), (b1, b2) = obtain_single_qubit_corrections(iswap_family_coords[3], (a, b, c), tau, Omega1, Omega2, delta)
    single_qubit_corrections['after_1'].append(a1)
    single_qubit_corrections['after_2'].append(a2)
    single_qubit_corrections['before_1'].append(b1)
    single_qubit_corrections['before_2'].append(b2)

>>> Using AshN-ND <<<
sinc_S1tau: 0.9958927352435614, sinc_S2tau: 0.9958927352435614
tau: 0.0080, Omega1: 0.0000, Omega2: 0.0000, delta: 0.0000
>>> Using AshN-ND <<<
sinc_S1tau: 0.9907728959880913, sinc_S2tau: 0.9907728959880913
tau: 0.0120, Omega1: 0.0000, Omega2: 0.0000, delta: 0.0000
>>> Using AshN-ND <<<
sinc_S1tau: 0.9836316430834658, sinc_S2tau: 0.9836316430834658
tau: 0.0160, Omega1: 0.0000, Omega2: 0.0000, delta: 0.0000
>>> Using AshN-ND <<<
sinc_S1tau: 0.9744953584044326, sinc_S2tau: 0.9744953584044326
tau: 0.0200, Omega1: 0.0000, Omega2: 0.0000, delta: 0.0000
>>> Using AshN-ND <<<
sinc_S1tau: 0.963397762004116, sinc_S2tau: 0.963397762004116
tau: 0.0240, Omega1: 0.0000, Omega2: 0.0000, delta: 0.0000
>>> Using AshN-ND <<<
sinc_S1tau: 0.9503797638053009, sinc_S2tau: 0.9503797638053009
tau: 0.0280, Omega1: 0.0000, Omega2: 0.0000, delta: 0.0000
>>> Using AshN-ND <<<
sinc_S1tau: 0.935489283788639, sinc_S2tau: 0.935489283788639
tau: 0.0320, Omega1: 0.0000, Omega2: 0.0000, delta: 0.0

In [21]:
for i, g in enumerate(iswap_family):
    console.rule('iSWAP-family gate {}:'.format(g))
    console.print(single_qubit_corrections['before_2'][i].round(3))


─────────────────────────────────────── iSWAP-family gate TK2(0.1, 0.1, 0): ───────────────────────────────────────

[[-0.   -0.j    -0.383+0.924j]
 [ 0.383+0.924j  0.   -0.j   ]]

────────────────────────────────────── iSWAP-family gate TK2(0.15, 0.15, 0): ──────────────────────────────────────

[[-0.   -0.j    -0.383+0.924j]
 [ 0.383+0.924j  0.   -0.j   ]]

─────────────────────────────────────── iSWAP-family gate TK2(0.2, 0.2, 0): ───────────────────────────────────────

...

[[-0.   -0.j    -0.383+0.924j]
 [ 0.383+0.924j  0.   -0.j   ]]

────────────────────────────────────── iSWAP-family gate TK2(0.45, 0.45, 0): ──────────────────────────────────────

[[-0.924-0.383j -0.   +0.j   ]
 [-0.   -0.j    -0.924+0.383j]]

─────────────────────────────────────── iSWAP-family gate TK2(0.5, 0.5, 0): ───────────────────────────────────────

[[-0.383+0.924j -0.   +0.j   ]
 [-0.   -0.j    -0.383-0.924j]]